In [1]:
% load_ext autoreload
% autoreload 2

UsageError: Line magic function `%` not found.


In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as ks, float32
from keras import backend as K

@tf.function
def get_coords_2d_former(map, camera):
    """ Creates a grid containing pixel coordinates normalized by the camera focal length """
    tf.print("map, cam", map, camera)
    b, h, w, c = map.get_shape().as_list()
    h_range = tf.range(0., h, 1.0, dtype=tf.float32) + 0.5
    w_range = tf.range(0., w, 1.0, dtype=tf.float32) + 0.5
    grid_x, grid_y = tf.meshgrid(w_range, h_range)
    mesh = tf.reshape(tf.stack([grid_x, grid_y], axis=2), [1, h, w, 2]) - tf.reshape(camera["c"], [b, 1, 1, 2])

    divide = tf.divide(mesh, tf.reshape(camera["f"], [b, 1, 1, 2]))

    coords_2d = tf.concat([divide, tf.ones([b,h,w,1])], axis=-1)
    tf.print("concat", coords_2d[0, 0, 0])
    coords_2d = tf.expand_dims(coords_2d, -1)
    return coords_2d, mesh

@tf.function
def repeat_const(tensor, myconst):
    shapes = tf.shape(tensor)
    return tf.repeat(myconst, shapes[0], axis=0)

@tf.function
def get_coords_2d(map, camera):
    """ Creates a grid containing pixel coordinates normalized by the camera focal length """
    # Modif test
    tf.print("map, cam", map, camera)
    h, w = map.get_shape().as_list()[1:3]
    h_range = tf.range(0., h, 1.0, dtype=tf.float32) + 0.5
    w_range = tf.range(0., w, 1.0, dtype=tf.float32) + 0.5

    new_cam_c = ks.layers.Reshape((1, 1, 2,), )(camera["c"])
    new_cam_f = ks.layers.Reshape((1, 1, 2,), )(camera["f"])
    print(new_cam_c)
    grid_x, grid_y = tf.meshgrid(w_range, h_range)
    mesh = tf.reshape(tf.stack([grid_x, grid_y], axis=2), [1, h, w, 2])

    mesh = mesh - new_cam_c
    divide = tf.divide(mesh, new_cam_f)

    myconst = tf.convert_to_tensor(np.ones((1, h, w, 1)).astype('float32'))
    ones_ = tf.keras.layers.Lambda(lambda x: repeat_const(x, myconst))(map)
    # Petit rappel pour te dire que tu as mis 1 journée à trouver ça

    coords_2d = ks.layers.Concatenate()([divide, ones_])
    coords_2d = tf.expand_dims(coords_2d, -1)
    return coords_2d, mesh


map_ = ks.Input(shape=(6, 6, 192,), dtype=float32)  # data image
camera = {"f": ks.Input(shape=(2,), dtype=float32),
          "c": ks.Input(shape=(2,), dtype=float32)}

depth_curr_l_function = lambda x: get_coords_2d(x[0], x[1])
depth_curr_l = ks.layers.Lambda(depth_curr_l_function)((map_, camera))

model_full = ks.Model(inputs=[map_, camera],
                      outputs=depth_curr_l)

b=1
map_test_1 = tf.random.uniform([b] + map_.shape[1:])
camera_test_1 = {"f":  tf.random.uniform([b] + camera["f"].shape[1:]),
          "c":  tf.random.uniform([b] + camera["c"].shape[1:])}

out_put_test_1,_ = model_full((map_test_1,camera_test_1))
print("---")
print()
out_put_test_1_former,_ = get_coords_2d_former(map_test_1,camera_test_1)
print("---")
print()
print()
print("map_test_1", map_test_1.shape)
print("out_put_test_1", out_put_test_1.shape, map_test_1.shape)
print("out_put_test_1_former", out_put_test_1_former.shape, map_test_1.shape)
print(out_put_test_1[0][0][0])
print(out_put_test_1_former[0][0][0])
print()
b=50
map_test_1 = tf.random.uniform([b] + map_.shape[1:])
camera_test_1 = {"f":  tf.random.uniform([b] + camera["f"].shape[1:]),
          "c":  tf.random.uniform([b] + camera["c"].shape[1:])}

out_put_test_1,_ = model_full((map_test_1,camera_test_1))
out_put_test_1_former,_ = get_coords_2d_former(map_test_1,camera_test_1)
print()
print("out_put_test_1", out_put_test_1.shape, map_test_1.shape)
print("out_put_test_1_former", out_put_test_1_former.shape, map_test_1.shape)
print()


Tensor("reshape/Reshape:0", shape=(None, 1, 1, 2), dtype=float32)
map, cam [[[[0.675126433 0.0856727362 0.237849951 ... 0.518973947 0.489841938 0.306459427]
   [0.700797677 0.567268 0.82176137 ... 0.789835334 0.322536349 0.0687612295]
   [0.234147549 0.839651942 0.0258088112 ... 0.134926796 0.103389263 0.576280236]
   [0.122982144 0.54301095 0.418167 ... 0.431989074 0.515849352 0.381710887]
   [0.19130373 0.497870207 0.637060761 ... 0.250590324 0.519442677 0.753737092]
   [0.301702261 0.90483284 0.130069375 ... 0.740432382 0.333977103 0.688129783]]

  [[0.19715941 0.340826869 0.739736557 ... 0.700846434 0.695834517 0.876043081]
   [0.227387428 0.909688473 0.604643345 ... 0.486281633 0.0739291906 0.318032265]
   [0.0998551846 0.757712364 0.399434447 ... 0.685925 0.84707582 0.174701691]
   [0.125960588 0.874973416 0.716504 ... 0.434343219 0.508990526 0.0751616955]
   [0.309620261 0.189978838 0.813502431 ... 0.876631 0.476808071 0.454391122]
   [0.71855545 0.171828866 0.712147117 ... 0.89